# Bootstrapped quantiles and standard errors for 90% confidence intervals

In [1]:
from sklearn.utils import resample
import pandas as pd
import numpy as np
import warnings

In [2]:
import rpy2.robjects.packages as rpackages
from rpy2.robjects.packages import importr
import rpy2.rinterface

In [3]:
KernSmooth = rpackages.importr('KernSmooth')
warnings.filterwarnings('ignore')

### Import and prepare data

In [4]:
def process_data(df, output_file):
    """This function adds squared and interaction terms to the Carneiro data set."""

    # Delete redundant columns\n",
    for key_ in ["newid", "caseid"]:
        del df[key_]

    # Add squared terms
    for key_ in ["mhgc", "cafqt", "avurate", "lurate_17", "numsibs", "lavlocwage17"]:
        str_ = key_ + "sq"
        df[str_] = df[key_] ** 2

    # Add interaction terms
    for j in ["pub4", "lwage5_17", "lurate_17", "tuit4c"]:
        for i in ["cafqt", "mhgc", "numsibs"]:
            df[j + i] = df[j] * df[i]

In [5]:
basic = pd.read_stata('data/basicvariables.dta')
local = pd.read_stata('data/localvariables.dta') 
df = pd.concat([basic, local], axis = 1)
process_data(df,'data/aer-replication-mock')

In [6]:
Z_list = ['const', 
 'cafqt',
 'cafqtsq',
 'mhgc',
 'mhgcsq',
 'numsibs',
 'numsibssq',
 'urban14',
 'lavlocwage17',
 'lavlocwage17sq',
 'avurate',
 'avuratesq',
 'd57',
 'd58',
 'd59',
 'd60',
 'd61',
 'd62',
 'd63',
 'lwage5_17numsibs',
 'lwage5_17mhgc',
 'lwage5_17cafqt',
 'lwage5_17',
 'lurate_17',
 'lurate_17numsibs',
 'lurate_17mhgc',
 'lurate_17cafqt',
 'tuit4c',
 'tuit4cnumsibs',
 'tuit4cmhgc',
 'tuit4ccafqt',
 'pub4',
 'pub4numsibs',
 'pub4mhgc',
 'pub4cafqt']

In [7]:
X_list = ['const',
'exp', 
'expsq', 
'lwage5', 
'lurate', 
'cafqt', 
'cafqtsq', 
'mhgc', 
'mhgcsq', 
'numsibs', 
'numsibssq', 
'urban14', 
'lavlocwage17', 
'lavlocwage17sq', 
'avurate', 
'avuratesq', 
'd57', 
'd58', 
'd59', 
'd60', 
'd61', 
'd62', 
'd63']

# Bootstrap algorithm

First, draw $B = 250$ independent bootstrap samples and compute the MTE* $ \ \hat{\theta}^{MTE}(b)$ for each bootstrap replication $b = 1,..,B$, where $B$ is the number of bootstrap samples used. Second, estimate the standard error of $\hat{\theta}^{MTE}$, denoted by $\widehat{\mathbf{se}}_B$, separately for each gridpoint $u_D$:

$$ \widehat{\mathbf{se}}_B = \frac{1}{B-1} \sum_{b = 1}^{B}{\{ \hat{\theta}}^{MTE}(b) \ - \ \hat{\theta}^{MTE}(\cdot)    \}^2 $$

where $\hat{\theta}^{MTE}(\cdot) = B^{-1} \sum_{b = 1}^{B}{ \hat{\theta}}^{MTE}(b)$.

With every iteration of the bootstrap, $\hat{P}(z)$ is reestimated, so the evaluation points $u_D$ over which to compute the *MTE* vary with each iteration. I compute the mean for each row of the 500 evaluation points that are reproduced with each bootstrap. 

In [8]:
# Number of iterations
B = 250

In [9]:
# Prepare empty arrays to store output values
mte_4_R = np.zeros([500, B])
quantiles_u_R = np.zeros([500, B])

In [10]:
for i in range(B): 
    boot = resample(df, replace=True, n_samples=len(df), random_state=None)

    # define Z and estimate propensity score via logit
    Z_boot = boot[Z_list]
    logitRslt_boot = sm.Logit(boot['state'], Z_boot).fit(disp=0)
    
    ps = logitRslt_boot.predict(Z_boot)
    
    # got an error before, had to change "boot['ps'] = ps" to the code below
    boot.insert(len(df.columns), 'ps', ps)
    
    treated = boot[['state', 'ps']][boot['state'] == 1]
    untreated = boot[['state', 'ps']][boot['state'] == 0]
    
    boot_trim = boot[(boot.ps >= np.min(treated['ps'])) & (boot.ps <= np.max(untreated['ps']))]
    ps_trim = ps[(ps >= np.min(treated['ps'])) & (ps <= np.max(untreated['ps']))]
    
    # sort by ps
    boot_trim = boot_trim.sort_values(by='ps', ascending=True)
    ps_trim = np.sort(ps_trim)
    ps_trim = pd.Series(ps_trim)
    
    X_trim = boot_trim[X_list]
    X_trim.reset_index(drop=True, inplace=True)
    
    # construct X_k * P(z)
    N = len(X_trim)
    P_z = pd.concat([ps_trim]*len(X_trim.columns), axis=1, ignore_index=True)
    Xp_trim= pd.DataFrame(X_trim.values*P_z.values, columns=[key_ + "_ps" for key_ in list(X_trim)], index=X_trim.index)
    
    
    Y_trim = boot_trim[['wage']]

    %reload_ext rpy2.ipython
        
    %R -i X_trim
    %R -i Xp_trim
    %R -i Y_trim
    %R -i ps_trim

    %R X <- as.matrix(X_trim)
    %R Xp <- as.matrix(Xp_trim)
    %R Y <- as.matrix(Y_trim)
    %R ps <- as.matrix(ps_trim)

    # Compute the lowess residuals of X, Xp and Y    
    %R res_Y_R <- loess(Y ~ ps, span = 0.05, degree = 1)$res
    %R res_X_R <- apply(X, 2, function(x) loess(x ~ ps, span = 0.05, degree = 1)$res)
    %R res_Xp_R <- apply(Xp, 2, function(x) loess(x ~ ps, span = 0.05, degree = 1)$res)

    %R res_Y_R <- data.frame(res_Y_R)
    %R res_X_R <- data.frame(res_X_R)
    %R res_Xp_R <- data.frame(res_Xp_R)

    %R -o res_Y_R
    %R -o res_X_R
    %R -o res_Xp_R

    # Append res_X and res_Xp. 
    res_X_Xp_R = np.append(res_X_R, res_Xp_R, axis=1)

    model_R = sm.OLS(res_Y_R, res_X_Xp_R)
    results_R = model_R.fit()
    b1_R = results_R.params[:len(list(X_trim))]
    b1_b0_R = results_R.params[len((list(X_trim))):]

    # prepare arrays
    ps_arr_R = np.array(ps_trim)
    X_arr_R = np.array(X_trim)
    Xp_arr_R = np.array(Xp_trim)
    Y_arr_R = boot_trim['wage']

    # compute the unobserved part of Y
    Y_tilde_R = Y_arr_R - np.dot(X_arr_R, b1_R) - np.dot(Xp_arr_R, b1_b0_R)
        
    %reload_ext rpy2.ipython
    
    %R -i Y_tilde_R
    %R -i ps_arr_R
    %R Y_tilde_R <- as.matrix(Y_tilde_R)
    %R ps_arr_R <- as.matrix(ps_arr_R)

    %R mte_u_poly_R <- locpoly(ps_arr_R, Y_tilde_R, drv = 1L, degree=2, bandwidth = 0.322, gridsize = 500L, range.x = c(0.005, 0.995))

    # Return the mte_u component and the quantiles of u_D as data frames and export them back into Python.
    %R mte_u_R <- mte_u_poly_R$y
    %R mte_u_R <- data.frame(mte_u_R)
    %R quantiles_R <- mte_u_poly_R$x
    %R quantiles_R <- data.frame(quantiles_R)

    %R -o mte_u_R
    %R -o quantiles_R
    
    # Turn data frames into arrays
    mte_u_R = np.array(mte_u_R)
    mte_u_R = np.concatenate(mte_u_R, axis=0)
    
    quantiles_R = np.array(quantiles_R)
    quantiles_R = np.concatenate(quantiles_R, axis=0)
    
    mte_x_R = np.dot(X_trim, b1_b0_R).mean(axis=0)
    mte_R = mte_x_R + mte_u_R
    
    mte_4_R[:,i] = mte_R/4
        
    quantiles_u_R[:,i] = quantiles_R

Calculate mean of quantiles $u_D$

In [11]:
quantiles_R_mean = quantiles_u_R.mean(axis=1)

In [12]:
# Turn into data frame to export as csv
quantiles_R_mean = pd.DataFrame(quantiles_R_mean)
quantiles_R_mean.to_csv(r'/Users/sebastiangsell/Documents/MSc_Economics_Bonn/Thesis/replication-semipar-carneiro2011/quantiles_R_mean.csv')

Compute separate standard error for each gridpoint $u_D$

In [13]:
mte_R_mean = mte_4_R.mean(axis=1)

no_sum = np.zeros([len(mte_4_R), B])

for row in range(len(mte_4_R)):
    for column in range(B):
        no_sum[row,column] = ((mte_4_R[row,column] - mte_R_mean[row])**2)

se_R = (1/(B-1)) * np.sum(no_sum, axis=1)

In [14]:
# Turn into data frame to export as csv
se_R = pd.DataFrame(se_R)
se_R.to_csv(r'/Users/sebastiangsell/Documents/MSc_Economics_Bonn/Thesis/replication-semipar-carneiro2011/se_R.csv')